In [ ]:
from collections import defaultdict
from glob import glob
from random import choice, sample


from tensorflow.keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D,Activation, Dropout, BatchNormalization,  GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Subtract,Multiply
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from skimage.io import imshow
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import cv2
import os
from tqdm import tqdm

In [ ]:
# from collections import defaultdict
# from glob import glob
# from random import choice, sample

# import cv2
# import numpy as np
# import pandas as pd
# from keras.applications.resnet50 import ResNet50, preprocess_input
# from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
# from keras.layers import Input, Dense, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract
# from keras.models import Model
# from keras.optimizers import Adam

# # File paths
# train_file_path = "../data/train/"
# train_relationships_path = "../csv_files/train_relationships.csv"
# validation_path = "F09"

# all_images = glob(train_folders_path + "*/*/*.jpg")

# train_images = [x for x in all_images if val_famillies not in x]
# val_images = [x for x in all_images if val_famillies in x]

# train_person_to_images_map = defaultdict(list)

# ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]

# for x in train_images:
#     train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

# val_person_to_images_map = defaultdict(list)

# for x in val_images:
#     val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

In [ ]:
# File paths
train_file_path = "train/"
train_relationships_path = "train_relationships.csv"
#the validation set will be family members F09...
validation_set = "F09"

#get all the images
all_images = glob(train_file_path + "*/*/*.jpg")

#seperate the train and validation sets
train_images = [img for img in all_images if validation_set not in img]
val_images = [img for img in all_images if validation_set in img]
    

In [ ]:
#create a dictionary with key=family member and value=list of pictures of family member
def fam_mem_pics(all_images):
    #create dictionary
    fam_member_dict = {}
    #create list for member pictures
    fam_member_lst = []
    i = 0
    while i < len(all_images)-1:
        #get the family member as key
        key = all_images[i][6:16]
        #check if picture is about the same fmaily member
        same_member = True

        while same_member:
            #check if same family member
            if key in all_images[i]:
                #append the image to the family member list
                fam_member_lst.append(all_images[i])
                
            else:
                #changed family member
                same_member = False
            
            #update the counter to not go out of bounds
            if (i+1) < len(all_images):  
                i += 1
            else:
                #break if out of bounds
                break
            
        #create key=family member and value= list of family member's pictures
        fam_member_dict[key] = fam_member_lst
        fam_member_lst = []
        
    #return the member_dictionary 
    return fam_member_dict


In [ ]:
#get a family member image dictionary
map_train_person_to_images = fam_mem_pics(train_images)
map_val_person_to_images = fam_mem_pics(val_images)

In [ ]:
map_val_person_to_images

In [ ]:
#get the raltionships dataframe
df = pd.read_csv("train_relationships.csv")

#get isolate the the family/member portions to compare
all_family_members = [img[6:16] for img in all_images]

#some relationships are not present within the relationship csv
#remove them by only keeping the ones mentioned in the dataset
df = df[df['p1'].isin(all_family_members)]
df = df[df['p2'].isin(all_family_members)]

In [ ]:
#seperate the training and validation labels
train = df[~df['p1'].str.contains(validation_set)]
val = df[df['p1'].str.contains(validation_set)]

#turn dataframes to tuples to get labels
train = list(zip(train['p1'].values, train['p2'].values))
val = list(zip(val['p1'].values, val['p2'].values))


In [ ]:
train

In [ ]:
train_person_to_images_map

In [ ]:
all_family_members

In [ ]:
def read_img(path):
    img = cv2.imread(path)
    return preprocess_input(img)


def gen(list_tuples, person_to_images_map, batch_size=16):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)
        print(batch_tuples)
        print(labels)
        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)
            print('p1: ', p1)
            print('p2: ', p2)
            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)
        print(batch_tuples)
        print(labels)
        for x in batch_tuples:
            print('x:, ', x)
            if not len(person_to_images_map[x[0]]):
                print('x[0]:, ', x)
                print(x[0])
        print('------------------------')
        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1 = np.array([read_img(x) for x in X1])

        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2 = np.array([read_img(x) for x in X2])

#         yield [X1, X2], labels

In [ ]:
gen(train, train_person_to_images_map, batch_size=16)

In [ ]:
gen(train, train_person_to_images_map, batch_size=16)

In [ ]:
def gen(list_tuples, person_to_images_map, batch_size=16):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)
        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)

            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)

        for x in batch_tuples:
            if not len(person_to_images_map[x[0]]):
                print(x[0])

        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1 = np.array([read_img(x) for x in X1])

        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2 = np.array([read_img(x) for x in X2])
        
        print(X1)
        
        yield [X1, X2], labels
        
gen(train, train_person_to_images_map, batch_size=16)

In [ ]:
# pip install opencv-python

In [ ]:
df = pd.read_csv('train_relationships.csv')
df.head()

In [ ]:
train_path = "train/"

families = os.listdir(train_path)


In [ ]:
from PIL import Image

In [ ]:
train_path = "train/"

for i in range(len(os.listdir(train_path)[:1])): #len(os.listdir(train_path))
    print(df.iloc[0])
    
    member_1 = os.listdir(train_path + str(df.iloc[0]['p1']))
    member_2 = os.listdir(train_path + str(df.iloc[0]['p2']))
    
    for member_pic in member_1
    
    print('-----------')
    print(member_1)
    print(member_2)
    

In [ ]:
from PIL import Image

In [ ]:
for face in member_1:
    try:  
        img  = Image.open('train/F0002/MID1/P00016_face2.jpg')  
    except:
        print('not work')
        pass

In [ ]:
img

In [ ]:
random_member_1_face = np.random.randint(low=0, high=len(member_1))
random_member_2_face = np.random.randint(low=0, high=len(member_2))

print(random_member_1_face)
print(random_member_2_face)

In [ ]:
from tensorflow.keras.preprocessing import image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

for face in member_1:
    
    filename = face
    print(filename)
#     img = image.load_img(filename, target_size=(150, 150))
# #     plt.imshow(img)
# #     plt.show()

#     img_tensor = image.img_to_array(img)
#     img_tensor = np.expand_dims(img_tensor, axis=0)

#     #Follow the Original Model Preprocessing
#     img_tensor /= 255.

#     #Check tensor shape
#     print(img_tensor.shape)

#     #Preview an image
#     plt.imshow(img_tensor[0])
#     plt.show()

In [ ]:
os.listdir(test)

In [ ]:
#Your code here
from keras.preprocessing.image import ImageDataGenerator

train_dir = 'train'
# validation_dir = 'data_org/val/'
test_dir = 'test'

# All images will be rescaled by 1./255
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

size = 255

train_generator = datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(size, size),
        color_mode="rgb"
        batch_size=20,
        class_mode='categorical',
        subset='training')

validation_generator = datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(size, size),
        color_mode="rgb"
        batch_size=20,
        class_mode='categorical',
        subset='validation')



In [ ]:
train_images, train_labels = next(train_generator)
val_images, val_labels = next(validation_generator)

In [ ]:
print(train_images.shape)
print(val_images.shape)

In [ ]:
print(train_labels.shape)
print(val_labels.shape)

In [ ]:
train_img = train_images.reshape(train_images.shape[0], -1)
print(train_img.shape)

In [ ]:
trai

In [ ]:
from tensorflow.keras.applications import VGG19

cnn_base = VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(size, size, 3))

cnn_base.summary()

In [ ]:
cnn_base.trainable = False

In [ ]:
# model = models.Sequential()
# model.add(cnn_base)
# model.add(layers.Flatten())
# # model.add(layers.Dense(64, activation='relu'))
# # model.add(layers.Dense(128, activation='relu'))
# # model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(120, activation='sigmoid'))

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(255, 255,  3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer="sgd",
              metrics=['acc'])

In [ ]:
# model.compile(loss='categorical_crossentropy',
#               metrics=['acc'])

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=4)